In [1]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

In [2]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/dataset/gen1/event_frame/single/base.yaml',
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/experiment/single/train.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
dynamically_modify_train_config(config=merged_conf)

num_class 2
Set partition sizes: (8, 10)


In [3]:
from modules.fetch import fetch_data_module, fetch_model_module

data = fetch_data_module(merged_conf)
data.setup('fit')
model = fetch_model_module(merged_conf)
model.setup('fit')

train dataset size: 186
valid dataset size: 1200
rvt
RVT
PAFPN
neck input channels (64, 128, 256)
head strides (8, 16, 32)
YOLOX-Head


In [4]:
import torch
ckpt_path = '../scripts/result/gen1/rvt-t/event_frame-dt50/20241116-134504/train/epoch=49-val_AP=0.42.ckpt'
ckpt = torch.load(ckpt_path, map_location='cpu')
model.load_state_dict(ckpt['state_dict'])

rnn_model = model.model
rnn_model.eval()

/var/folders/0f/5nhrfybd7r1d725z2nywhzg00000gn/T/ipykernel_64976/3261947062.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cp

RVTYOLOX(
  (backbone): RVT(
    (stages): ModuleList(
      (0): RVTStage(
        (downsample_cf2cl): ConvDownsampling_Cf2Cl(
          (conv): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3), bias=False)
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (att_blocks): ModuleList(
          (0): MaxVitAttentionPairCl(
            (att_window): PartitionAttentionCl(
              (norm1): Identity()
              (self_attn): SelfAttentionCl(
                (qkv): Linear(in_features=32, out_features=96, bias=True)
                (proj): Linear(in_features=32, out_features=32, bias=True)
              )
              (ls1): LayerScale()
              (drop_path1): Identity()
              (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
              (mlp): MLP(
                (net): Sequential(
                  (0): Sequential(
                    (0): Linear(in_features=32, out_features=128, bias=True)
           

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from utils.yolox_utils import to_yolox, postprocess
from functools import partial

# Postprocessの設定
post_process = partial(postprocess, num_classes=2, conf_thre=0.45, nms_thre=0.1, class_agnostic=False)

# 推論モード設定
mode = 'val'
prev_rnn_state = None
loop_count = 1200  # 最大ループ回数
current_loop = 0

# プロットするループ回数を指定
plot_intervals = [250, 500, 1050]
plot_results = {}  # プロット用に保存する辞書 {ループ回数: (画像, 推論結果)}

# 色変換関数
def change_colors(image):
    """
    赤 (ONイベント) を緑に、青 (OFFイベント) を黄に、灰色 (背景) を白に変換。
    :param image: RGB画像 (H, W, C)
    :return: 色変換後の画像 (H, W, C)
    """
    # 変換先の色
    color_mapping = {
        (255, 0, 0): (255, 0, 0),   # 赤 -> ピンク
        (0, 0, 255): (0, 0, 255),   # 青 -> 水色
        (114, 114, 114): (0, 0, 0)  # 灰色 -> 白
    }
    
    # ベクトル化して色変換を効率化
    reshaped_image = image.reshape(-1, image.shape[-1])  # 2次元に展開
    mapped_image = np.array([color_mapping.get(tuple(pixel), tuple(pixel)) for pixel in reshaped_image])
    return mapped_image.reshape(image.shape)

# 推論時の勾配計算を無効化
with torch.no_grad():
    # dataloader のループ
    for batch in data.val_dataloader():
        if current_loop >= loop_count:  # 最大ループ回数を超えたら終了
            break

        # イベントとラベルを取得
        events = batch['events'][:, 0].float()  # 最初のシーケンス
        labels = batch['labels']

        # ラベルを YOLOX の形式に変換
        targets = to_yolox(labels, mode=mode)[:, 0]

        # RNNモデルを使用して推論
        outputs, state = rnn_model(events, prev_rnn_state)

        # 推論結果を postprocess
        processed_pred = post_process(outputs)

        # RNNの状態を保存
        prev_rnn_state = state

        # 特定のループ回数に達したら結果を保存
        if current_loop in plot_intervals:
            event_image = batch['events'][0, 0].numpy()
            event_image = np.transpose(event_image, (1, 2, 0))  # (C, H, W) -> (H, W, C)
            plot_results[current_loop] = (event_image, processed_pred)

        current_loop += 1

# プロット
for loop_num, (event_image, processed_pred) in plot_results.items():
    # 色変換処理を適用
    converted_image = change_colors(event_image.astype(np.uint8))
    
    plt.figure(figsize=(10, 8))
    plt.imshow(converted_image)  # 色変換後の画像を表示

    # バウンディングボックスをプロットする場合
    if processed_pred and processed_pred[0] is not None:
        predictions = processed_pred[0]  # (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
        for bbox in predictions:
            x1, y1, x2, y2, obj_conf, class_conf, class_pred = bbox.numpy()
            # バウンディングボックスを描画
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2 - x1, y2 - y1, edgecolor='yellow', facecolor='none', linewidth=5))
            # クラスとスコアを表示
            plt.text(x1, y1 - 5, f'Class: {int(class_pred)}, Conf: {obj_conf:.2f}', 
                     color='yellow', fontsize=10, bbox=dict(facecolor='black', alpha=0.5))
    else:
        print(f"No detections found at loop {loop_num}.")

    # plt.title(f"Output at Loop {loop_num}")
    plt.axis('off')
    plt.gca().set_axis_off()  # 軸の周辺を非表示に
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # 余白を削除
    plt.savefig(f"dt5_{loop_num}.png", bbox_inches='tight', pad_inches=0)  # 余白なく保存
    plt.close()  # プロットを閉じる
    # plt.axis('off')
    # plt.show()


In [6]:
# import matplotlib.pyplot as plt
# import numpy as np
# from utils.yolox_utils import to_yolox, postprocess
# from functools import partial

# # Postprocessの設定
# post_process = partial(postprocess, num_classes=2, conf_thre=0.45, nms_thre=0.1, class_agnostic=False)

# # 推論モード設定
# mode = 'val'
# prev_rnn_state = None
# loop_count = 1200  # 最大ループ回数
# current_loop = 0

# # プロットするループ回数を指定
# plot_intervals = [250, 500, 1050]
# plot_results = {}  # プロット用に保存する辞書 {ループ回数: (画像, 推論結果)}

# # 推論時の勾配計算を無効化
# with torch.no_grad():
#     # dataloader のループ
#     for batch in data.val_dataloader():
#         if current_loop >= loop_count:  # 最大ループ回数を超えたら終了
#             break

#         # イベントとラベルを取得
#         events = batch['events'][:, 0].float()  # 最初のシーケンス
#         labels = batch['labels']

#         # ラベルを YOLOX の形式に変換
#         targets = to_yolox(labels, mode=mode)[:, 0]

#         # RNNモデルを使用して推論
#         outputs, state = rnn_model(events, prev_rnn_state)

#         # 推論結果を postprocess
#         processed_pred = post_process(outputs)

#         # RNNの状態を保存
#         prev_rnn_state = state

#         # 特定のループ回数に達したら結果を保存
#         if current_loop in plot_intervals:
#             event_image = batch['events'][0, 0].numpy()
#             event_image = np.transpose(event_image, (1, 2, 0))  # (C, H, W) -> (H, W, C)
#             plot_results[current_loop] = (event_image, processed_pred)

#         current_loop += 1

# # プロット
# for loop_num, (event_image, processed_pred) in plot_results.items():
#     plt.figure(figsize=(10, 8))
#     plt.imshow(event_image)  # RGB画像をそのまま表示

#     # バウンディングボックスをプロットする場合
#     if processed_pred and processed_pred[0] is not None:
#         predictions = processed_pred[0]  # (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
#         for bbox in predictions:
#             x1, y1, x2, y2, obj_conf, class_conf, class_pred = bbox.numpy()
#             # バウンディングボックスを描画
#             plt.gca().add_patch(plt.Rectangle(
#                 (x1, y1), x2 - x1, y2 - y1, edgecolor='yellow', facecolor='none', linewidth=2))
#             # クラスとスコアを表示
#             plt.text(x1, y1 - 5, f'Class: {int(class_pred)}, Conf: {obj_conf:.2f}', 
#                      color='yellow', fontsize=10, bbox=dict(facecolor='black', alpha=0.5))
#     else:
#         print(f"No detections found at loop {loop_num}.")

#     # plt.title(f"Output at Loop {loop_num}")
#     plt.axis('off')
#     plt.gca().set_axis_off()  # 軸の周辺を非表示に
#     plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # 余白を削除
#     plt.savefig(f"dt100_{loop_num}.png", bbox_inches='tight', pad_inches=0)  # 余白なく保存
#     plt.close()  # プロットを閉じる
#     # plt.axis('off')
#     # plt.show()


In [7]:
# import matplotlib.pyplot as plt
# import numpy as np
# from utils.yolox_utils import to_yolox, postprocess
# from functools import partial

# # Postprocessの設定
# post_process = partial(postprocess, num_classes=2, conf_thre=0.45, nms_thre=0.1, class_agnostic=False)

# # 推論モード設定
# mode = 'val'
# prev_rnn_state = None
# loop_count = 500  # ループ回数を指定
# current_loop = 0
# final_batch = None
# final_processed_pred = None

# # 推論時の勾配計算を無効化
# with torch.no_grad():
#     # dataloader のループ
#     for batch in data.val_dataloader():
#         if current_loop >= loop_count:  # 指定した回数に達したら終了
#             break

#         # イベントとラベルを取得
#         events = batch['events'][:, 0].float()  # 最初のシーケンス
#         labels = batch['labels']

#         # ラベルを YOLOX の形式に変換
#         targets = to_yolox(labels, mode=mode)[:, 0]

#         # RNNモデルを使用して推論
#         outputs, state = rnn_model(events, prev_rnn_state)

#         # 推論結果を postprocess
#         processed_pred = post_process(outputs)

#         # RNNの状態を保存
#         prev_rnn_state = state

#         # 最後のループのバッチと推論結果を保存
#         final_batch = batch
#         final_processed_pred = processed_pred

#         current_loop += 1

# # 最後の画像をプロット
# event_image = final_batch['events'][0, 0].numpy()  # 計算グラフがないのでそのまま numpy() 使用可能
# event_image = np.transpose(event_image, (1, 2, 0))  # (C, H, W) -> (H, W, C) に変換

# plt.figure(figsize=(10, 8))
# plt.imshow(event_image)  # RGB画像をそのまま表示

# # バウンディングボックスをプロットする場合
# if final_processed_pred and final_processed_pred[0] is not None:
#     predictions = final_processed_pred[0]  # (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
#     for bbox in predictions:
#         # そのまま numpy() 使用
#         x1, y1, x2, y2, obj_conf, class_conf, class_pred = bbox.numpy()
#         # バウンディングボックスを描画
#         plt.gca().add_patch(plt.Rectangle(
#             (x1, y1), x2 - x1, y2 - y1, edgecolor='yellow', facecolor='none', linewidth=2))
#         # クラスとスコアを表示
#         plt.text(x1, y1 - 5, f'Class: {int(class_pred)}, Conf: {obj_conf:.2f}', 
#                  color='yellow', fontsize=10, bbox=dict(facecolor='black', alpha=0.5))
# else:
#     print("No detections found in the final loop.")

# plt.title("Final Output After 100 Loops")
# plt.axis('off')
# plt.show()
